In [1]:
##Import the necessary libraries
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import (
    Input,
    DepthwiseConv2D,
    Conv2D,
    BatchNormalization,
    ReLU,
    GlobalAveragePooling2D,
    Flatten,
    Dense,
    Dropout,
    Activation,
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model

from keras import backend as K

from keras.models import Model
from keras.layers import (
    DepthwiseConv2D,
    Conv2D,
    BatchNormalization,
    AveragePooling2D,
    Dense,
    Activation,
    Flatten,
    Reshape,
    Add,
    Dropout,
    Input,
)
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
import numpy as np


def depthwise_block(inputs, stride, kernel_sz=(3, 3), pad="same"):
    """
    Function that performs the depthwise convolution
    Inputs:
        inputs:    the input shape of the depthwise convolution
        kernel_sz: a tuple that indicates the size of the filtering kernel
        stride:    a tuple that indicates the strides of the kernel
    Return:
        x: the result of the depthwise convolution
    """

    x = DepthwiseConv2D(
        kernel_size=kernel_sz, strides=stride, depth_multiplier=1, padding=pad
    )(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation="relu")(x)

    return x


def pointwise_block(
    inputs,
    num_filters,
    alpha,
    kernel_sz=(1, 1),
    stride=(1, 1),
    pad="same",
    dropout=False,
    dropout_perc=0.1,
):
    """
    Function that performs the pointwise convolution
    Inputs:
        inputs:      the input shape of the depthwise convolution
        num_filters: number of filters to be used in the convolution
        kernel_sz:   a tuple that indicates the size of the filtering kernel
        stride:      a tuple that indicates the strides of the kernel
        dropout: determine if dropout will be performed
        dropout_perc: percentage of neurons that will be set to zero
    Return:
        x: the result of the pointwise convolution
    """

    # Number of filters based on width multiplier reported in the original paper
    n_fil = int(num_filters * alpha)

    x = Conv2D(filters=n_fil, kernel_size=kernel_sz, padding=pad)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation="relu")(x)

    if dropout == True:
        x = Dropout(dropout_perc)(x)

    return x


def MobileNetV1(
    input_shape,
    num_units,
    filters=32,
    kernel_sz=(3, 3),
    stride=(2, 2),
    alp=1,
    ro=1,
    dropout_perc=0.1,
):
    input_shape = (int(input_shape[0] * ro), int(input_shape[1] * ro), input_shape[2])

    inputs = Input(shape=input_shape)

    # Regular convolution
    x = Conv2D(filters=filters, kernel_size=kernel_sz, strides=stride)(inputs)
    x = BatchNormalization()(x)
    x = Activation(activation="relu")(x)
    x = Dropout(dropout_perc)(x)

    # First depthwise-pointwise block
    x = depthwise_block(x, kernel_sz=(3, 3), stride=(1, 1))
    x = pointwise_block(
        x,
        num_filters=64,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    # Second depthwise-pointwise block
    x = depthwise_block(x, kernel_sz=(3, 3), stride=(2, 2))
    x = pointwise_block(
        x,
        num_filters=128,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    # Third depthwise-pointwise block
    x = depthwise_block(x, kernel_sz=(3, 3), stride=(1, 1))
    x = pointwise_block(
        x,
        num_filters=128,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    # Fourth depthwise-pointwise block
    x = depthwise_block(x, kernel_sz=(3, 3), stride=(2, 2))
    x = pointwise_block(
        x,
        num_filters=256,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    # Fifth depthwise-pointwise block
    x = depthwise_block(x, kernel_sz=(3, 3), stride=(1, 1))
    x = pointwise_block(
        x,
        num_filters=256,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    # Sixth depthwise-pointwise block
    x = depthwise_block(x, kernel_sz=(3, 3), stride=(2, 2))
    x = pointwise_block(
        x,
        num_filters=512,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    # Seventh depthwise-pointwise block (repeated five times)
    x = depthwise_block(x, kernel_sz=(3, 3), stride=(1, 1))
    x = pointwise_block(
        x,
        num_filters=512,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    x = depthwise_block(x, kernel_sz=(3, 3), stride=(1, 1))
    x = pointwise_block(
        x,
        num_filters=512,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    x = depthwise_block(x, kernel_sz=(3, 3), stride=(1, 1))
    x = pointwise_block(
        x,
        num_filters=512,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    x = depthwise_block(x, kernel_sz=(3, 3), stride=(1, 1))
    x = pointwise_block(
        x,
        num_filters=512,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    x = depthwise_block(x, kernel_sz=(3, 3), stride=(1, 1))
    x = pointwise_block(
        x,
        num_filters=512,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    # Eight depthwise-pointwise block
    x = depthwise_block(x, kernel_sz=(3, 3), stride=(2, 2))
    x = pointwise_block(
        x,
        num_filters=1024,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    # Nineth depthwise-pointwise block
    x = depthwise_block(x, kernel_sz=(3, 3), stride=(1, 1))
    x = pointwise_block(
        x,
        num_filters=1024,
        alpha=alp,
        stride=(1, 1),
        dropout=True,
        dropout_perc=dropout_perc,
    )

    # Pooling layer
    # Pooling size correction due to the resolution multiplier parameter
    pool_size = int(np.round(7 * ro))
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    x = Flatten()(x)

    # Fully connected layer
    x = Dense(units=1024, activation="relu")(x)

    # Softmax layer
    output = Dense(num_units, activation="softmax")(x)

    model = Model(inputs, output)

    return model


### Training on CIFAR-10

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 2s 0us/step


In [3]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train, dtype ="uint8")
y_test = to_categorical(y_test, dtype ="uint8")

Results from various values of alpha

$α = 1$

In [4]:
alpha = 1
ro = 1

model_v1 = MobileNetV1(
    (32,32,3),
    num_units=10,
    alp=alpha,
    ro=ro,
    dropout_perc=0.2)

optimizer = Adam(learning_rate=0.001)

model_v1.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=10,
    verbose=1,
    min_lr=0.0001
)

history_train = model_v1.fit(
    x_train,
    y_train, 
    batch_size=64,
    epochs=10, 
    validation_data=(
        x_test, 
        y_test
    )
)

Epoch 1/10
782/782 [==============================] - 613s 779ms/step - loss: 2.1248 - accuracy: 0.1824 - val_loss: 2.4367 - val_accuracy: 0.1614
Epoch 2/10
782/782 [==============================] - 597s 763ms/step - loss: 1.8284 - accuracy: 0.2838 - val_loss: 2.3285 - val_accuracy: 0.2565
Epoch 3/10
782/782 [==============================] - 597s 763ms/step - loss: 1.6559 - accuracy: 0.3656 - val_loss: 2.1112 - val_accuracy: 0.3578
Epoch 4/10
782/782 [==============================] - 598s 765ms/step - loss: 1.5472 - accuracy: 0.4179 - val_loss: 1.5785 - val_accuracy: 0.4550
Epoch 5/10
782/782 [==============================] - 613s 784ms/step - loss: 1.4737 - accuracy: 0.4613 - val_loss: 1.7193 - val_accuracy: 0.4495
Epoch 6/10
782/782 [==============================] - 597s 763ms/step - loss: 1.3902 - accuracy: 0.5010 - val_loss: 1.3083 - val_accuracy: 0.5410
Epoch 7/10
782/782 [==============================] - 597s 764ms/step - loss: 1.3134 - accuracy: 0.5363 - val_loss: 1.3573 -

$α = 0.75$

In [6]:
alpha = 0.75
ro = 1

model_v1 = MobileNetV1(
    (32,32,3),
    num_units=10,
    alp=alpha,
    ro=ro,
    dropout_perc=0.2)

optimizer = Adam(learning_rate=0.001)

model_v1.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=10,
    verbose=1,
    min_lr=0.0001
)

history_train = model_v1.fit(
    x_train,
    y_train, 
    batch_size=64,
    epochs=10, 
    validation_data=(
        x_test, 
        y_test
    )
)

Epoch 1/10
782/782 [==============================] - 338s 428ms/step - loss: 2.1648 - accuracy: 0.1728 - val_loss: 2.4458 - val_accuracy: 0.1567
Epoch 2/10
782/782 [==============================] - 334s 427ms/step - loss: 1.8840 - accuracy: 0.2709 - val_loss: 2.0636 - val_accuracy: 0.2596
Epoch 3/10
782/782 [==============================] - 334s 428ms/step - loss: 1.7528 - accuracy: 0.3189 - val_loss: 1.8737 - val_accuracy: 0.3224
Epoch 4/10
782/782 [==============================] - 333s 426ms/step - loss: 1.6403 - accuracy: 0.3728 - val_loss: 1.7832 - val_accuracy: 0.3887
Epoch 5/10
782/782 [==============================] - 333s 426ms/step - loss: 1.5438 - accuracy: 0.4260 - val_loss: 1.5852 - val_accuracy: 0.4437
Epoch 6/10
782/782 [==============================] - 332s 425ms/step - loss: 1.4503 - accuracy: 0.4694 - val_loss: 1.4768 - val_accuracy: 0.4795
Epoch 7/10
782/782 [==============================] - 333s 426ms/step - loss: 1.3687 - accuracy: 0.5063 - val_loss: 1.3243 -

$α = 0.5$

In [7]:
alpha = 0.5
ro = 1

model_v1 = MobileNetV1(
    (32,32,3),
    num_units=10,
    alp=alpha,
    ro=ro,
    dropout_perc=0.2)

optimizer = Adam(learning_rate=0.001)

model_v1.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=10,
    verbose=1,
    min_lr=0.0001
)

history_train = model_v1.fit(
    x_train,
    y_train, 
    batch_size=64,
    epochs=10, 
    validation_data=(
        x_test, 
        y_test
    )
)

Epoch 1/10
782/782 [==============================] - 173s 216ms/step - loss: 2.1962 - accuracy: 0.1600 - val_loss: 3.0149 - val_accuracy: 0.0997
Epoch 2/10
782/782 [==============================] - 167s 214ms/step - loss: 1.9347 - accuracy: 0.2413 - val_loss: 2.6844 - val_accuracy: 0.1627
Epoch 3/10
782/782 [==============================] - 163s 209ms/step - loss: 1.7962 - accuracy: 0.3024 - val_loss: 2.1809 - val_accuracy: 0.2746
Epoch 4/10
782/782 [==============================] - 169s 217ms/step - loss: 1.6956 - accuracy: 0.3421 - val_loss: 1.8871 - val_accuracy: 0.3094
Epoch 5/10
782/782 [==============================] - 173s 221ms/step - loss: 1.6207 - accuracy: 0.3860 - val_loss: 1.7658 - val_accuracy: 0.3778
Epoch 6/10
782/782 [==============================] - 172s 220ms/step - loss: 1.5494 - accuracy: 0.4228 - val_loss: 1.4709 - val_accuracy: 0.4569
Epoch 7/10
782/782 [==============================] - 172s 219ms/step - loss: 1.4803 - accuracy: 0.4586 - val_loss: 1.6370 -

$α = 0.25$

In [8]:
alpha = 0.25
ro = 1

model_v1 = MobileNetV1(
    (32,32,3),
    num_units=10,
    alp=alpha,
    ro=ro,
    dropout_perc=0.2)

optimizer = Adam(learning_rate=0.001)

model_v1.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=10,
    verbose=1,
    min_lr=0.0001
)

history_train = model_v1.fit(
    x_train,
    y_train, 
    batch_size=64,
    epochs=10, 
    validation_data=(
        x_test, 
        y_test
    )
)

Epoch 1/10
782/782 [==============================] - 93s 115ms/step - loss: 2.2450 - accuracy: 0.1419 - val_loss: 2.5673 - val_accuracy: 0.1164
Epoch 2/10
782/782 [==============================] - 89s 114ms/step - loss: 2.0509 - accuracy: 0.1885 - val_loss: 1.9715 - val_accuracy: 0.2169
Epoch 3/10
782/782 [==============================] - 89s 114ms/step - loss: 1.9359 - accuracy: 0.2365 - val_loss: 1.9941 - val_accuracy: 0.2457
Epoch 4/10
782/782 [==============================] - 89s 114ms/step - loss: 1.8529 - accuracy: 0.2734 - val_loss: 2.1046 - val_accuracy: 0.2697
Epoch 5/10
782/782 [==============================] - 91s 116ms/step - loss: 1.7967 - accuracy: 0.2992 - val_loss: 1.7206 - val_accuracy: 0.3259
Epoch 6/10
782/782 [==============================] - 88s 113ms/step - loss: 1.7363 - accuracy: 0.3244 - val_loss: 1.8616 - val_accuracy: 0.3002
Epoch 7/10
782/782 [==============================] - 89s 114ms/step - loss: 1.6859 - accuracy: 0.3473 - val_loss: 1.6580 - val_ac